In [1]:
import numpy as np
from collections import deque
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint

from coremdlr.viz import model_plots, CorePlotter
from coremdlr.models import NetworkModel
from coremdlr.datasets import FaciesDataset
from coremdlr.config.defaults import WELL_NAMES

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
wavenet_model_args = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 20,
        'block_filters' : 128,
        'residual_filters' : 64,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    },
    'loss' : {'categorical_crossentropy': 0.5,
              'ordinal_squared_error': 0.5}
}

CHKPT_FNAME = 'temp_best_weights.h5'

wavenet_fit_args = {
    'batch_size' : 16,
    'epochs' : 15,
    'class_weighted' : True
}

names_deq = deque(WELL_NAMES)

In [3]:
results = {}

for i in range(len(names_deq) // 3):
    
    # Set up dataset, rotate well_names
    current_names = list(names_deq)
    fdset = FaciesDataset(current_names[:-3], test_wells=current_names[-3:],
                        features=["pseudoGR", "logs"],
                        pseudoGR_args={'features': ['mean', 'var'], 
                                                    'per_channel' : True},
                        label_resolution=32)
    
    names_deq.rotate(3)
    
    print(fdset)
    fdset.load_or_generate_data()
    
    # Build and train and re-load weights
    wnet_model = NetworkModel(fdset, model_args=wavenet_model_args)
    
    
    chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_acc', verbose=1,
                                    save_best_only=True, 
                                    save_weights_only=True)
    
    wavenet_fit_args['callbacks'] = [chkpt_callback]
    
    wnet_model.fit(fdset, **wavenet_fit_args)
    
    wnet_model.network.load_weights(CHKPT_FNAME)
    
    os.remove(CHKPT_FNAME)
    
    # Get test results
    for test_well in fdset.test_well_names:
        
        results[test_well] = wnet_model.preds_dataframe(test_well, logs=['GR', 'RDEP', 'PEF', 'SP'])


Facies Patch Dataset loaded from 
Features: ['logs', 'pseudoGR']
Classes: ['none', 'bad-sandstone', 'sandstone', 'clay-prone sandstone', 'sandy mudstone', 'oilstained', 'mudstone']
Train Wells: ['204-19-2', '204-19-3A', '204-19-6', '204-19-7', '204-20-1', '204-20-1Z', '204-20-2', '204-20-3', '204-20-6a', '204-20a-7']
Test Wells: ['204-24a-6', '204-24a-7', '205-21b-3']

Loading Well:  204-19-2  from  /home/administrator/Dropbox/core_data/facies/train_data


/home/administrator/code/python/coremdlr/coremdlr/facies/datasets/utils.py:72: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/administrator/code/python/coremdlr/coremdlr/facies/datasets/utils.py:76: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))
Header section Parameter regexp=~P was not found.


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (2683,)), ('top', (2683,)), ('base', (2683,)), ('pseudoGR', (2683, 32, 8)), ('logs', (2683, 11))]
Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8)), ('logs', (9590, 11))]
Loading Well:  204-19-6  from  /home/administrator/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']


...

Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('pseudoGR', (3542, 32, 8)), ('logs', (3542, 11))]
Loading Well:  204-20a-7  from  /home/administrator/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('pseudoGR', (3105, 32, 8)), ('logs', (3105, 11))]
Shapes of `(batch_X, batch_y)`: (16, 4096, 8), (16, 128, 5)
Epoch 1/15
73/74 [============================>.] - ETA: 0s - loss: 5.1021 - acc: 0.1278Epoch 1/15

Epoch 00001: val_acc improved from -inf to 0.08695, saving model to temp_best_weights.h5
74/74 [==============================] - 51s 685ms/step - loss: 5.1020 - acc: 0.1268 - val_loss: 4.9730 - val_acc: 0.0870
Epoch 2/15
73/74 [============================>.] - ETA: 0s - loss: 4.4839 - acc: 0.1707
Epoch 00002: val_acc improved from 0.08695 to 0.56108, saving model to temp_best_weights.h5
74/74 [==============================] - 29s 392ms/step - loss: 4.4608 - acc: 0.1742 - val_loss: 2.5339 - val_acc: 0.5611
Epoch 3/15
73/74 [============================>.] - ETA: 0s - loss: 2.1097 - acc: 0.5391
Epoch 00003: val_acc did not improve from 0.56108
74/74 [==========

ValueError: need at least one array to concatenate

In [ ]:
import pandas as pd

dfs = []
for well_name, results_df in results.items():
    results_df['well_name'] = well_name
    dfs.append(results_df)
    
df = pd.concat(dfs, ignore_index=True)
df

In [ ]:
df.to_csv('cross_test_wavenet_all_data.csv', index=False)

In [ ]:
axes = df.regression.hist(by=df.y_true, figsize=(15,15), alpha=0.4)
print(axes)

for i, ax in enumerate(axes.flat):
    df.regression[df.y_pred==i].hist(ax=ax, color='blue', alpha=0.4)

In [ ]:
axes = df.y_true.hist(by=df.well_name, figsize=(15,15), alpha=0.4)

for i, (ax, name) in enumerate(zip(axes.flat, df.WELL_NAME)):
    print(name)
    df.y_pred[df.well_name==name].hist(ax=ax, color='blue', alpha=0.4)

In [ ]:
fdset.well_names

In [ ]:
fdset.test_well_names

In [ ]:
np.arange(30).reshape(5, 3, 2)

In [ ]:
np.arange(30).reshape(5, 3, 2).reshape(-1, 2).reshape(5, 3 ,2)